In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
data=pd.read_csv(r"D:/MLAI/dataset/Brain Stroke.csv")

In [3]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [4]:
data.shape

(4981, 11)

In [5]:
data.isna().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [6]:
data.stroke.value_counts()

0    4733
1     248
Name: stroke, dtype: int64

In [7]:
x=data.iloc[:,[1,2,3,7,8]]
y=data.iloc[:,-1]

# Data Imbalancing

In [8]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ro=RandomOverSampler()
x_resample,y_resample=ro.fit_resample(x,y)
print(Counter(y_resample))

Counter({1: 4733, 0: 4733})


# Feature Scaling by StandardScaler()


In [9]:
# Feature Scaling by StandardScaler()
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x_data=ss.fit_transform(x_resample)
x_data

array([[ 0.53171127, -0.46182101,  2.75355206,  1.98206843,  1.16705102],
       [ 1.11276875, -0.46182101,  2.75355206, -0.23049499,  0.51466365],
       [-0.27282985, -0.46182101, -0.36316728,  0.94652315,  0.8169895 ],
       ...,
       [ 0.48701454, -0.46182101, -0.36316728,  0.5848212 , -0.2809307 ],
       [ 1.06807202,  2.16534106,  2.75355206, -1.04112547, -0.07407617],
       [ 1.02337529, -0.46182101, -0.36316728, -1.04599141, -0.34457825]])

In [10]:
y_data=y_resample

# slpit data into train and test


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=100)


In [12]:
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 3792, 1: 3780})
Counter({1: 953, 0: 941})


# Model Imbalancing--> Cross_validation


In [13]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,random_state=100,shuffle=True)
kf.get_n_splits(x_data)
print(kf)

KFold(n_splits=5, random_state=100, shuffle=True)


In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
l1=LogisticRegression()

In [15]:
#without Iteration based cross validation
scores=cross_val_score(l1,x_train,y_train,cv=kf)
y_pred=cross_val_predict(l1,x_test,y_test,cv=kf)
print(scores)
print(y_pred)

[0.76369637 0.75709571 0.78335535 0.78137384 0.77873184]
[0 1 1 ... 0 0 1]


In [16]:
kx_train_data=[]
kx_test_data=[]
ky_train_data=[]
ky_test_data=[]
for train_data,test_data in kf.split(x_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=kf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=kf)
    kx_train_data.append(x_train)
    kx_test_data.append(x_test)
    ky_train_data.append(y_train)
    ky_test_data.append(y_test)
    print("Checking Balancing y test........",Counter(y_test))
    print(y_pred)
    print(scores)
    print(np.mean(scores)*100)


Checking Balancing y test........ Counter({1: 953, 0: 941})
[1 0 1 ... 0 1 1]
[0.7669967  0.76963696 0.76486129 0.76420079 0.7985469 ]
77.2848529238657
Checking Balancing y test........ Counter({0: 969, 1: 924})
[1 1 1 ... 1 1 1]
[0.76237624 0.7689769  0.76237624 0.77675033 0.75693527]
76.54829947988193
Checking Balancing y test........ Counter({1: 957, 0: 936})
[1 1 1 ... 1 1 1]
[0.78745875 0.74455446 0.78415842 0.74768824 0.77873184]
76.85183392843908
Checking Balancing y test........ Counter({0: 947, 1: 946})
[1 1 1 ... 1 1 1]
[0.78085809 0.78481848 0.76567657 0.77146631 0.7681638 ]
77.41966508407776
Checking Balancing y test........ Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.76435644 0.76237624 0.77755776 0.75759577 0.79524439]
77.1426117512676


In [17]:
for train_data,test_data in kf.split(x_data,y_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=kf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=kf)
    print("Checking y_test Balancing:",Counter(y_test))
    print("Checking y_train Balancing:",Counter(y_train))
    print(scores)
    print("-------------------------------------------------------")

Checking y_test Balancing: Counter({1: 953, 0: 941})
Checking y_train Balancing: Counter({0: 3792, 1: 3780})
[0.7669967  0.76963696 0.76486129 0.76420079 0.7985469 ]
-------------------------------------------------------
Checking y_test Balancing: Counter({0: 969, 1: 924})
Checking y_train Balancing: Counter({1: 3809, 0: 3764})
[0.76237624 0.7689769  0.76237624 0.77675033 0.75693527]
-------------------------------------------------------
Checking y_test Balancing: Counter({1: 957, 0: 936})
Checking y_train Balancing: Counter({0: 3797, 1: 3776})
[0.78745875 0.74455446 0.78415842 0.74768824 0.77873184]
-------------------------------------------------------
Checking y_test Balancing: Counter({0: 947, 1: 946})
Checking y_train Balancing: Counter({1: 3787, 0: 3786})
[0.78085809 0.78481848 0.76567657 0.77146631 0.7681638 ]
-------------------------------------------------------
Checking y_test Balancing: Counter({1: 953, 0: 940})
Checking y_train Balancing: Counter({0: 3793, 1: 3780})
[0.

In [18]:
# 0: 955, 1: 42  --> 94.82
# 0: 944, 1: 52  --> 95.08
# 0: 954, 1: 42  --> 94.83
# 0: 935, 1: 61  --> 95.30
# 0: 945, 1: 51 --> 95.05

# StratifiedKFold Cross Validation

In [19]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5,random_state=100,shuffle=True)
skf.get_n_splits(x_data,y_data)
print(skf)

StratifiedKFold(n_splits=5, random_state=100, shuffle=True)


In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
l1=LogisticRegression()

In [21]:
scores=cross_val_score(l1,x_train,y_train,cv=skf)
y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
print(scores)
print(y_pred)

[0.78349835 0.75577558 0.78019802 0.7661823  0.78071334]
[0 1 1 ... 1 1 1]


In [22]:
kx_train_data=[]
kx_test_data=[]
ky_train_data=[]
ky_test_data=[]
for train_data,test_data in skf.split(x_data,y_data):
    x1_train,x1_test=x_data[train_data],x_data[test_data]
    y1_train,y1_test=y_data[train_data],y_data[test_data] 
    scores=cross_val_score(l1,x_train,y_train,cv=skf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
    kx_train_data.append(x_train)
    kx_test_data.append(x_test)
    ky_train_data.append(y_train)
    ky_test_data.append(y_test)
    print("Checking Balancing.......:",Counter(y_test))
    print(y_pred)
    print(scores)
    print(np.mean(scores)*100)

    

Checking Balancing.......: Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.78349835 0.75577558 0.78019802 0.7661823  0.78071334]
77.32735175763284
Checking Balancing.......: Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.78349835 0.75577558 0.78019802 0.7661823  0.78071334]
77.32735175763284
Checking Balancing.......: Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.78349835 0.75577558 0.78019802 0.7661823  0.78071334]
77.32735175763284
Checking Balancing.......: Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.78349835 0.75577558 0.78019802 0.7661823  0.78071334]
77.32735175763284
Checking Balancing.......: Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.78349835 0.75577558 0.78019802 0.7661823  0.78071334]
77.32735175763284


In [23]:
d1=pd.Series.to_frame(pd.Series(kx_test_data))
d2=pd.Series.to_frame(pd.Series(ky_test_data))

In [24]:
for train_data,test_data in skf.split(x_data,y_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=skf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
    print("Checking y_test Balancing:",Counter(y_test))
    print("Checking y_train Balancing: ",Counter(y_train))
    print(scores)
    print("---------------------------------------------------------")

Checking y_test Balancing: Counter({1: 947, 0: 947})
Checking y_train Balancing:  Counter({1: 3786, 0: 3786})
[0.77887789 0.78811881 0.76287979 0.77014531 0.77014531]
---------------------------------------------------------
Checking y_test Balancing: Counter({1: 947, 0: 946})
Checking y_train Balancing:  Counter({0: 3787, 1: 3786})
[0.78679868 0.77227723 0.75445545 0.75957728 0.76089828]
---------------------------------------------------------
Checking y_test Balancing: Counter({1: 947, 0: 946})
Checking y_train Balancing:  Counter({0: 3787, 1: 3786})
[0.7669967  0.78349835 0.78151815 0.76486129 0.74900925]
---------------------------------------------------------
Checking y_test Balancing: Counter({0: 947, 1: 946})
Checking y_train Balancing:  Counter({1: 3787, 0: 3786})
[0.75907591 0.7650165  0.77689769 0.77014531 0.78335535]
---------------------------------------------------------
Checking y_test Balancing: Counter({0: 947, 1: 946})
Checking y_train Balancing:  Counter({1: 3787, 

In [25]:
from sklearn.model_selection import ShuffleSplit
sf=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
sf.get_n_splits(x_data,y_data)

5

In [26]:
for train_data,test_data in sf.split(x_data,y_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=sf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
    print("Checking y_test Balancing:",Counter(y_test))
    print("Checking y_train Balancing:",Counter(y_train))
    print(np.mean(scores)*100)
    print(scores)
    print("---------------------------")

Checking y_test Balancing: Counter({1: 950, 0: 944})
Checking y_train Balancing: Counter({0: 3789, 1: 3783})
77.32013201320133
[0.77359736 0.77755776 0.78151815 0.7689769  0.76435644]
---------------------------
Checking y_test Balancing: Counter({0: 954, 1: 940})
Checking y_train Balancing: Counter({1: 3793, 0: 3779})
75.26072607260727
[0.77623762 0.74653465 0.75577558 0.74983498 0.73465347]
---------------------------
Checking y_test Balancing: Counter({0: 962, 1: 932})
Checking y_train Balancing: Counter({1: 3801, 0: 3771})
77.95379537953797
[0.76963696 0.77623762 0.79075908 0.77821782 0.78283828]
---------------------------
Checking y_test Balancing: Counter({0: 988, 1: 906})
Checking y_train Balancing: Counter({1: 3827, 0: 3745})
78.57425742574257
[0.78481848 0.78943894 0.78547855 0.78085809 0.78811881]
---------------------------
Checking y_test Balancing: Counter({0: 968, 1: 926})
Checking y_train Balancing: Counter({1: 3807, 0: 3765})
77.53135313531352
[0.77293729 0.76765677 0.

In [27]:
kf.split(x)

<generator object _BaseKFold.split at 0x0000022A9A0AC120>

In [28]:
skf.split(x,y)

<generator object _BaseKFold.split at 0x0000022A9A0AC7B0>